In [1]:
## Load Folder Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive/')

## Move Pointer File ke DATA_UM
%cd /content/drive/MyDrive/DATA_UM

## Declare Root Project File
root_file = "/content/drive/MyDrive/DATA_UM"

Mounted at /content/drive/
/content/drive/MyDrive/DATA_UM


In [2]:
import pandas as pd
import json
import csv
import re
import string
import numpy as np

In [3]:
## Load Preprocess SLANG Module

# Load Dataframe
df_tmp_nasalsabila = pd.read_csv('data/slang/colloquial-indonesian-lexicon_nasalsabila.csv')
df_slang_fendiirfan = pd.read_csv('data/slang/colloquial-indonesian-lexicon_fendiirfan.csv')
df_slang_custom = pd.read_csv('data/slang/custom_dictonary_slang.csv')
df_slang_custom_exception = pd.read_csv('data/slang/custom_dictonary_slang_dont_swap.csv')

# Transform and Delete Duplicate Data from nasalsabila
df_slang_nasalsabila = df_tmp_nasalsabila[['slang', 'formal']]
df_slang_nasalsabila.drop_duplicates(inplace=True)

# Transform and Delete Duplicate Data from fendiirfan
df_slang_fendiirfan.dropna(inplace=True)
df_slang_fendiirfan.rename(columns={'kataAlay': 'slang', 'kataBaik': 'formal'}, inplace=True)
df_slang_fendiirfan.drop_duplicates(inplace=True)

# Transform and Delete Duplicate our own data.
df_slang_custom.drop_duplicates(inplace=True)
df_slang_custom_exception.drop_duplicates(inplace=True)

def remove_slang(text, df_tmp):
    # To Lowercase
    text = text.lower()

    # Pecah to array
    words = text.split(" ");

    # DF to Array
    arr_slang = df_tmp['slang'].to_numpy()
    arr_formal = df_tmp['formal'].to_numpy()
    do_not_change = df_slang_custom_exception['kata'].to_numpy()

    # Array New Text
    new_text = []

    for kata in words:
      if kata in arr_slang:
        if kata not in do_not_change:
          idx = np.where(arr_slang==kata)[0][0]
          kata = arr_formal[idx]

      new_text.append(kata)
      text_new = ' '.join(new_text)

    return text_new

<ipython-input-3-188833ac7026>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slang_nasalsabila.drop_duplicates(inplace=True)


In [4]:
## Load Preprocess Stemming Modul
# Load and Download Library Sastrawi
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Indonesian Stopword by Sastrawi
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.0 MB/s eta 0:00:00


In [5]:
## Load Preprocess Stopword Data

# Load Data Stopword from Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# Load Data Stopword from Satya and Custom
stopword_satya = list(pd.read_csv('data/stopword/stopwords_satya.txt', header = None)[0])
custom_stopword = list(pd.read_csv('data/stopword/custom_stopword.txt', header = None)[0])

# Declare Function / Modul Stopword
def remove_stopwords(text):
  ## Stopword Removal from data by Sastrawi and Satwa
  text = stopword.remove(text)

  ## Stopword Removal from data by Satwa and Custom Word
  # Removal Stopword
  text_split = text.split(' ')
  temp_text_split = []
  for i in range(len(text_split)):
    ## Remove Stopword by Satwa : gak dipake dulu ini terlalu ngilangin konteks text
    #if (text_split[i] not in stopword_satya) and (text_split[i] not in custom_stopword):
    if (text_split[i] not in custom_stopword):
      temp_text_split.append(text_split[i])
  text = ' '.join(temp_text_split)

  return text

In [6]:
def clean_text_predict(text) :

    ## TEXT TRANSFORM
    # Remove username 1st
    text = re.sub('@[^\s]+','',text)
    # Remove hashtag dan @
    ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    # Change "-" to space
    text = text.replace("-", " ")
    # Remove punctiation
    text  = "".join([char for char in text if char not in string.punctuation])
    # Remove angka
    text = re.sub('[0-9]+', '', text)
    # Remove website link
    text = re.sub(r'http\S+', '', text)
    # Remove new line
    text = re.sub("\n"," ",text)
    # Transform to lowercase
    text = text.lower()
    # Replace perulangan huruf pada kalimat "iyaaa" to "iya"
    text = re.sub(r'(\w)(\1{2,})', r"\1", text)
    # will replace single word
    text = re.sub(r"\b[a-zA-Z]\b","",text)
    # will replace space more than one
    text = re.sub('( {2,})',' ',text)

    ## SLANG
    text = remove_slang(text ,df_slang_nasalsabila)
    text = remove_slang(text ,df_slang_fendiirfan)
    text = remove_slang(text ,df_slang_custom)

    ## STEMMING
    text = stemmer.stem(text)

    ## STOPWORD
    text = remove_stopwords(text)

    ## TEXT TRANSFORM BUT AFTER
    # will replace single word
    # text = re.sub(r"\b[a-zA-Z]\b","",text)

    return text

In [8]:
df_2022_proses = pd.read_excel('mentionlytics/all_data_under_2022_predicted_non_official_pre-processed.xlsx', index_col=0)
df_2022_proses

,date,platform,brand,hashtag,name,username,profile_id,comments,share_retweet,bookmark,quotes,likes,link,text,predicted,labeled,preprocessed
0,2021-08-30 00:00:00,Twitter,cimory,cimoryyogurt,makcellyahh🤭,@amikmooie,1253223231483343104,0,0,0,0,0,https://twitter.com/amikmooie/status/143231210...,Rasa banana ini lucu jugaak🤭🤭😚😚 #cimoryyogurt,Positive,NaN,rasa banana lucu jugaak cimoryyogurt
1,2019-12-18 00:00:00,Twitter,cimory,cimoryyogurt,bukan siapa-siapa,@Nabilaftr2,1624285875923484928,0,0,0,0,1,https://twitter.com/Nabilaftr2/status/12072840...,Sayangi ususmu minum cimory tiap hari.. Ehh😂 #...,Negative,1.0,sayang usus minum cimory tiap hari ehh jangann...
2,2017-08-14 00:00:00,Twitter,cimory,cimoryyogurt,Mumun Munawaroh,@mumunmanawaroh,860373179180605440,0,0,0,0,0,https://twitter.com/mumunmanawaroh/status/8971...,Yogurt dulu😉 #cimory #cimoryyogurt #enakbanget...,Negative,1.0,yogurt dulu cimory cimoryyogurt enakbanget
3,2017-07-04 00:00:00,Twitter,cimory,cimoryyogurt,wiThamee,@Withanissa,1177766821560143872,0,0,0,0,0,https://twitter.com/Withanissa/status/88224114...,..biar adem 😬 . . . . #yoghurt #cimoryyogurt #...,Neutral,NaN,adem yoghurt cimoryyogurt cimoryfreshmilk
4,2016-12-13 00:00:00,Twitter,cimory,cimoryyogurt,Indah Sucihati,@indah247_AS,737631410148646912,0,0,0,0,0,https://twitter.com/indah247_AS/status/8086224...,Rasanyaaa.....😋 . . . . . . . . . . #frisianfl...,Negative,1.0,rasa frisianflag coconutdelight strawberrymilk...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4084,2020-09-10 03:45:05,Instagram,milklife,#TheMilkofLife,GlobalFood,globalfoodid,17849789426318800,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,Pasti Seger nih susu nya 😁,Positive,NaN,segar susu
4085,2020-09-10 03:40:28,Instagram,milklife,#TheMilkofLife,ZakirZavier | MASAK | KULINER,abangadeksukamam,17862431987101570,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,wah oke bener nih susunya,Neutral,NaN,oke benar susu
4086,2020-09-10 03:38:53,Instagram,milklife,#TheMilkofLife,Food+Lifestyle Blogger | Vera,verajj,17881808839761910,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,😍😍😍 rekomen nih susu nya,Neutral,NaN,rekomen susu
4087,2020-09-10 03:38:39,Instagram,milklife,#TheMilkofLife,STEVEN LUKITA,makansamakoko,17887995268678600,0,0,0,0,0,https://www.instagram.com/p/CE8OoRtAXud/,Sehat ya jamesss,Positive,NaN,sehat james


In [9]:
df_2022_proses['preprocessed'] = df_2022_proses['text'].apply(lambda x: clean_text_predict(x))

In [18]:
df_2022_proses.to_excel('mentionlytics/all_data_under_2022_predicted_non_official_pre-processed_1.xlsx')

In [23]:
import re

def remove_non_full_url(text):
    # Regular expression to match URLs without protocol, subdomain, and directories
    url_pattern = re.compile(r'\b(?:https?://)?(?:www\.)?([a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?)\b')

    # Replace non-full URLs with an empty string
    return url_pattern.sub('', text)

# Example usage
tweet_text = "Check out my website at domain.com. Also, visit http://www.example.com/path/to/page for more information. #website #example bima. putra instagr.am/p/PMFWExw8ln/"
filtered_tweet = remove_non_full_url(tweet_text)
print(filtered_tweet)


Check out my website at . Also, visit  for more information. #website #example bima. putra /
